In [7]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [8]:
from langchain_openai import ChatOpenAI
proxy_picker_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, base_url="https://reverse.onechats.top/v1")

In [9]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "proxy_picker"

In [ ]:
object_fetching_prompt = ""
attribute_extraction_prompt = ""
interaction_parsing_prompt = ""
interaction_classification_prompt = ""
candidate_evaluation_prompt = ""

In [43]:
from langchain_core.messages import HumanMessage
import base64
import httpx

image_url = "http://localhost:8000/1.jpg"
#run the following command in the terminal for accessing local images
#cd /Users/jamaman/Documents/GitHub/ProxyPicker/images
#python3 -m http.server

image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the content in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = proxy_picker_llm.invoke([message])
print(response.content)

The image features a variety of items arranged on a flat surface. Here’s a description of the contents:

1. **Headphones**: A pair of over-ear headphones with a light brown cushion.
2. **Fitness Tracker**: A black wristband device, likely a fitness tracker.
3. **Mouse**: A black computer mouse with a glowing logo.
4. **Cup**: A metallic cup with a reflective surface, partially filled with liquid.
5. **Face Mask**: A white disposable face mask.
6. **Remote Control**: A white remote control with buttons and a digital display.
7. **Guitar Pick**: A red guitar pick.
8. **Duck Lamp**: A small, white duck-shaped lamp with yellow feet.
9. **Sticky Notes**: A green sticky note pad.
10. **Scissors**: A pair of white-handled scissors.
11. **Game Controller**: A gray game controller, likely for a gaming console.
12. **Pen**: A black pen.
13. **Tissue**: A crumpled tissue.
14. **Black Folder**: A small black folder or case.

These items suggest a workspace or a casual environment, possibly for gam